# 8장 감성 분석에 머신 러닝 적용

## 8.1 텍스트 처리용 Imdb 영화 리뷰 데이터 준비

- 긍정 또는 부정으로 레이블되어 있는 영화 리뷰 5만 개로 구성되어 있다. (긍정은 별 6개 이상, 부정은 별 5개 이하)

### 8.1.1 영화 리뷰 데이터셋 구하기
- https://ai.stanford.edu/~amaas/data/sentiment

### 8.1.2 영화 리뷰 데이터셋을 더 간편한 형태로 전처리

In [3]:
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

In [4]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

In [6]:
import pyprind
import pandas as pd
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는
# 디렉토리로 바꾸세요

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:18


In [7]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [8]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## 8.2 BoW 모델 소개
- 텍스트를 수치 특성 벡터로 표현하는 BoW(Bag-of-Word)를 소개한다.
1. 전체 문서에 대해 고유한 토큰, 예를 들어 단어로 이루어진 어휘 사전을 만든다.
2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만든다.  
  
  
### 8.2.1 단어를 특성 벡터로 변환
CountVectorizer는 문서 또는 문장으로 이루어진 텍스트 데이터 배열을 입력받아 BoW 모델을 만든다.

In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [10]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [11]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### 8.2.2 tf-idf를 사용하여 단어 적합성 평가
- 일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않다.
- 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법인 tf-idf를 배운다.
- tf-idf는 단어 빈도와 역문서 빈도의 곱으로 정의된다.  
    $tf-idf(t,d)=tf(t,d)\times idf(t,d)$  
    $idf(t,d) = log\frac{n_d} {1+df(d,t)} $  
  
CountVectorizer 클래스에서 만든 단어 빈도를 입력받아 tf-idf로 변환하는 TfidfTransformer 클래스가 구현되어 있다.

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


tf-idf를 계산하기 전에 단어 빈도를 정규화하지만, TfidfTransformer 클래스는 tf-idf를 직접 정규화한다.  
사이킷런의 TfidTransformer는 기본적으로 L2 정규화를 적용한다.  
$v_{norm} = \frac{v}{||v||_2} = \frac {v}{\sqrt {v_1^2+v_2^2+...+v_n^2}} = \frac{v}{(\sum^n _{i=1} v_i^2)^{1/2}} $   



### 8.2.3 텍스트 데이터 정제
- 불필요한 문자를 삭제하여 텍스트 데이터를 정제해야한다.
- 아래에서 확인할 수 있듯이 HTML 마크업은 물론 구두점과 글자가 아닌 문자가 포함되어 있다.
- HTML 마크업에는 유용한 의미가 많지 않지만 구두점은 특정 NLP 문제에서 쓸모 있는 추가 정보가 될 수 있다.

In [13]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

파이썬의 정규 표현식 라이브러리 re를 사용하여 작업을 수행한다.

In [14]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # HTML 마크업 삭제
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', '')) # 1개 단어 제외, 소문자로 변환
    return text

In [15]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [16]:
df['review'] = df['review'].apply(preprocessor)

### 8.2.4 문서를 토큰으로 나누기
 - 토큰화 하는 한 가지 방법은 공백 문자를 기준으로 개별 단어로 나누는 것이다.
 - 단어를 변하지 않는 기본 형태인 어간으로 바꾸는 어간 추출이라는 방법도 있다.
 - 파이썬의 NLTK 패키지에 포터 어간 추출 알고리즘이 구현되어 있다.

In [17]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [18]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [19]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

#### 불용어(stop-word) 제거
불용어는 모든 종류의 텍스트에 아주 흔하게 등장하는 단어이다.  
문서의 종류를 구별하는 데 사용할 수 있는 정보가 없거나 아주 조금만 있다.  
불용어의 예로는 is, and, has, like 등이 있다.  
불용어 제거는 tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 더 유용하다.

In [20]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [21]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]


['runner', 'like', 'run', 'run', 'lot']

## 8.3 문서 분류를 위한 로지스틱 회귀 모델 훈련
- 25000개씩 훈련 데이터셋과 테스트 데이터셋으로 나눈다.
- GridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 로지스틱 회귀 모델에 대한 최적의 매개변수 조합을 찾는다.

In [22]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [26]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x00000207A1088550>} 
CV 정확도: 0.897


In [27]:
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

테스트 정확도: 0.899


## 8.4 대용량 데이터 처리 : 온라인 알고리즘과 외부 메모리 학습

- 대량의 데이터셋을 다룰 수 있는 외부 메모리 학습 기법을 사용한다.
- 이 방법은 데이터셋을 작은 배치로 나누어 분류기를 점진적으로 학습시킨다.


In [29]:
import numpy as np
import re
from nltk.corpus import stopwords


# `stop` 객체를 앞에서 정의했지만 이전 코드를 실행하지 않고
# 편의상 여기에서부터 코드를 실행하기 위해 다시 만듭니다.
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

# 한 번에 문서 하나씩 읽어서 반환한다.
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [30]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [31]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

외부 메모리 학습에 CountVectorizer 클래스를 사용할 수 없다.  
이 클래스는 전체 어휘 사전을 메모리에 가지고 있어야 하기 때문이다.  
또 TfidfVectorizer 클래스는 역문서 빈도를 계산하기 위해 훈련 데이터셋의 특성 벡터를 모두 메모리에 가지고 있어야 한다.  
사이킷런에서 텍스트 처리에 사용할 수 있는 다른 유용한 클래스는 HaghingVectorizer이다.

In [32]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [33]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log', random_state=1)


doc_stream = stream_docs(path='movie_data.csv')

In [34]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


In [35]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [36]:
clf = clf.partial_fit(X_test, y_test)

## 8.5 잠재 디리클레 할당을 사용한 토픽 모델링
- 토핑 모델링은 레이블이 없는 텍스트 문서에 토픽을 할당하는 광범위한 분야이다.
- 예를 들어 대량의 뉴스 기사 데이터셋을 분류하는 일과 같은 경우이다.
- 토픽 모델링 애플리케이션은 이런 기사에 카테고리 레이블을 할당한다. 
- 인기 있는 토픽 모델링 기법인 잠재 디리클레 할당(LDA)를 소개한다.

### 8.5.1 LDA를 사용한 텍스트 문서 분해
- LDA 이면의 수학은 꽤 복잡하고 베이지안 추론에 관한 지식이 필요하다.
- LDA는 여러 문서에 걸쳐 자주 등장하는 단어의 그룹을 찾는 확률적 생성 모델이다.
- 각 문서를 여러 단어가 혼합된 것으로 가정하면 토픽은 자주 등장하는 단어들로 나타낼 수 있다.
- LDA는 입력으로 받은 BoW 행렬을 두 개의 행렬로 분해해야한다.
- 문서-토픽 행렬 ,단어-토픽 행렬
- 이 두 행렬을 곱해서 가능한 작은 오차로 BoW 입력 행렬을 재구성할 수이ㄸ로고 LDA가 BoW를 분해한다.
- 실제로는 LDA가 BoW 행렬에서 찾은 토픽이 관심 대상인 것이다. 유일한 단점은 토픽 개수를 정해야 한다는 것이다.

### 8.5.2 사이킷런의 LDA
이 절에서 사이킷런에 구현된 LatentDirichletAlloction 클래스를 사용하여 영화 리뷰 데이터 셋을 분해하고 열 개의 토픽으로 분류한다.  


In [37]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [40]:
from sklearn.feature_extraction.text import CountVectorizer

# max_df 최대 문서 빈도를 지정 여기서는 10%(0.1)
count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [41]:
from sklearn.decomposition import LatentDirichletAllocation
# 문서에서 10개의 토픽을 추정하도록 학습 (n_components)
lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [42]:
lda.components_.shape

(10, 5000)

In [43]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


C:\Users\chang\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [44]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
